# Tweeter

In [1]:
import tweepy
import datetime

ModuleNotFoundError: No module named 'tweepy'

In [ ]:

# Authenticate to Twitter
CONSUMER_KEY = 'YOUR_API_KEY'
CONSUMER_SECRET = 'YOUR_API_SECRET_KEY'
ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN'
ACCESS_TOKEN_SECRET = 'YOUR_ACCESS_TOKEN_SECRET'

auth = tweepy.OAuth1UserHandler(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth)

# Define date range
end_date = datetime.datetime.now() 
start_date = end_date - datetime.timedelta(days=7)

# Search for tweets about Apple from the past week
tweets = tweepy.Cursor(api.search, q="Apple", since=start_date, until=end_date, lang="en").items()

# Print tweets
for tweet in tweets:
    print(tweet.text)


# Reddit

In [39]:
import praw
from datetime import datetime, timedelta
import pandas as pd
import csv
import time

#### set up token and test

In [4]:

# Authentication
reddit = praw.Reddit(
    client_id="bOUU3GVPswoqHAeWY-WB5g",
    client_secret="65G2ItcdbqJZU9pmGBC6303fYV-QJg",
    user_agent="script:sentimental_analysis:v1.0 (by /u/Legal_Advertising127)"
  #  username=''
)



In [21]:
subreddit = reddit.subreddit("python")

In [22]:
top_posts = subreddit.top(limit=10)
new_posts = subreddit.new(limit=10)

In [23]:
for post in top_posts:
    print('Title - ', post.title)
    print('ID - ', post.id)
    print('Author - ', post.author)
    print('URL - ', post.url)
    print('Score - ', post.score)
    print('Comment count - ', post.num_comments)
    print("Created -", post.created_utc)
    print("\n")
    
    

Title -  Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.
ID -  g53lxf
Author -  iEslam
URL -  https://i.redd.it/2s0dj8ob12u41.png
Score -  12341
Comment count -  133
Created - 1587424299.0


Title -  This post has:
ID -  hoolsm
Author -  Krukerfluk
URL -  https://www.reddit.com/r/Python/comments/hoolsm/this_post_has/
Score -  9237
Comment count -  437
Created - 1594386373.0


Title -  I redesign the Python logo to make it more modern
ID -  gftejm
Author -  jessjwilliamson
URL -  https://i.redd.it/rxezjyf4ojx41.png
Score -  7865
Comment count -  266
Created - 1588945149.0


Title -  Automate the boring stuff with python - tinder
ID -  7kpme8
Author -  backprop88
URL -  https://gfycat.com/PointlessSimplisticAmericanquarterhorse
Score -  6719
Comment count -  328
Created - 1513644476.0


Title -  Just finished programming and building my own smart mirror in python, wrote all of the code myself and implemented my own voice control and facial recog

In [11]:
for post in new_posts:
    print('Title - ', post.title)
    print('ID - ', post.id)
    print('Author - ', post.author)
    print('URL - ', post.url)
    print('Score - ', post.score)
    print('Comment count - ', post.num_comments)
    print("Created -", post.created_utc)
    print("\n")
    

Title -  Looking for contributors for my open-source AI security project
ID -  18d3rdq
Author -  kanxx030
URL -  https://www.reddit.com/r/Python/comments/18d3rdq/looking_for_contributors_for_my_opensource_ai/
Score -  1
Comment count -  0
Created - 1701978353.0


Title -  Pygolo 0.2.0: write Python extensions in Go!
ID -  18d3nt3
Author -  cavokz
URL -  https://www.reddit.com/r/Python/comments/18d3nt3/pygolo_020_write_python_extensions_in_go/
Score -  1
Comment count -  0
Created - 1701978092.0


Title -  CPython Dynamic Dispatch Internals: What Happens When You do a + b?
ID -  18d0axx
Author -  abhi9u
URL -  https://codeconfessions.substack.com/p/cpython-dynamic-dispatch-internals
Score -  2
Comment count -  0
Created - 1701969063.0


Title -  [Video] Explaining Floats Imprecision 🤖
ID -  18d05dh
Author -  JosephLovesPython
URL -  https://www.reddit.com/r/Python/comments/18d05dh/video_explaining_floats_imprecision/
Score -  0
Comment count -  0
Created - 1701968647.0


Title -  GitHub

#### collect data

In [40]:
# Calculate the UNIX timestamp for one week ago
one_week_ago = datetime.now() - timedelta(days=7)
timestamp = int(one_week_ago.timestamp())


# List of stock market-related subreddits
subreddits = ["stocks", "investing", "wallstreetbets", "StockMarket", "options", "SecurityAnalysis", "Daytrading"]

# Search term
search_term = "Apple"

# Basic rate limiting
def rate_limited_request(subreddits, term):

    data = []
    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        try:
            for post in subreddit.search(term, time_filter="week"):
                if post.created_utc >= timestamp:  # Ensure posts are only from the last week

                    # Convert the post's timestamp to the desired format (day month year)
                    post_time = datetime.utcfromtimestamp(post.created_utc).strftime('%d %B %Y')
                    
                    # Add post title and body to the data list
                    data.append([post.title + "\n" + post.selftext, post_time, subreddit_name])
                    
                    try:
                        post.comments.replace_more(limit=0)  # Fetch top-level comments only
                        for comment in post.comments:
                            comment_time = datetime.utcfromtimestamp(comment.created_utc).strftime('%d %B %Y')
                            data.append([comment.body, comment_time, subreddit_name])
                    except praw.exceptions.RedditAPIException as e:
                        print(f"Error fetching comments: {e}")
                        time.sleep(10)  # Sleep for 10 seconds before retrying
        except praw.exceptions.RedditAPIException as e:
            print(f"Rate limit exceeded: {e}")
            time.sleep(10)  # Sleep for 10 seconds before retrying
    return data


data = rate_limited_request(subreddits, "Apple")


In [45]:
len(data[0])

3

In [46]:
df = pd.DataFrame(data, columns=["Text", "Date", "Section"])


In [49]:
df.head()

,Text,Date,Section
0,"For the first time since August, Apple's marke...",06 December 2023,stocks
1,>the iPhone maker’s stock price has risen over...,06 December 2023,stocks
2,"Jeeez, feels like yesterday when they hit a tr...",06 December 2023,stocks
3,"…”soars **to**”, otherwise the implication is ...",06 December 2023,stocks
4,Look at India’s iPhone share and get back to m...,06 December 2023,stocks


In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



In [57]:

# Your sample text
text = df.iloc[3]["Text"]

# Tokenize the text
tokens = word_tokenize(text)

# Get English stop words
stop_words = set(stopwords.words('english'))

# Filter out the stop words and non-letter tokens
filtered_text = [word for word in tokens if word.isalpha() and word.lower() not in stop_words]

# Rejoin filtered text
filtered_sentence = ' '.join(filtered_text)

print(filtered_sentence)

soars otherwise implication soars


In [56]:
df.iloc[3]["Text"]

'…”soars **to**”, otherwise the implication is “soars by”…'

In [63]:
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


# Function to get wordnet POS tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def clean_text(text):
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Get English stop words
    stop_words = set(stopwords.words('english'))

    # Filter out the stop words, non-letter tokens, and lemmatize
    filtered_text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens if word.isalpha() and word.lower() not in stop_words]

    # Rejoin filtered text
    filtered_sentence = ' '.join(filtered_text)

    return filtered_sentence

In [64]:
clean_text(df.iloc[4]["Text"])

'Look India iPhone share get back Services fast grow segment profit raise price Apple One translate instal device grow like past year still plenty room'

In [67]:
df['Text'] = df['Text'].apply(clean_text)

In [68]:
df.iloc[4]["Text"]

'Look India iPhone share get back Services fast grow segment profit raise price Apple One translate instal device grow like past year still plenty room'

In [70]:
df.to_csv('reddit_data.csv')

In [73]:
df_load = pd.read_csv('reddit_data.csv', index_col=0)

In [74]:
df_load

,Text,Date,Section
0,first time since August Apple market capitaliz...,06 December 2023,stocks
1,iPhone maker stock price risen year even compa...,06 December 2023,stocks
2,Jeeez feel like yesterday hit trillion,06 December 2023,stocks
3,soar otherwise implication soar,06 December 2023,stocks
4,Look India iPhone share get back Services fast...,06 December 2023,stocks
...,...,...,...
1271,Fintech Apple wave goodbye Goldman Sachs credi...,05 December 2023,SecurityAnalysis
1272,Looking advice long SQQQ HXD position Hi Every...,04 December 2023,Daytrading
1273,man leveraged ETFs settle daily meant held lon...,04 December 2023,Daytrading
1274,Looking advice long SQQQ HXD position Hi Every...,03 December 2023,Daytrading


In [65]:
# Calculate the UNIX timestamp for one week ago
one_week_ago = datetime.now() - timedelta(days=7)
timestamp = int(one_week_ago.timestamp())

# List to store the data
data = []

subreddit = reddit.subreddit("all")
for post in subreddit.search("Apple", time_filter="week"):
    if post.created_utc >= timestamp:  # Ensure posts are only from the last week

        # Convert the post's timestamp to the desired format (day month year)
        post_time = datetime.utcfromtimestamp(post.created_utc).strftime('%d %B %Y')
        
        # Add post title and body to the data list
        data.append([post.title + "\n" + post.selftext, post_time])
        
        post.comments.replace_more(limit=None)  # Fetch more comments
        for comment in post.comments.list():
            comment_time = datetime.utcfromtimestamp(comment.created_utc).strftime('%d %B %Y')
            data.append([comment.body, comment_time])

# Save data to a CSV file
with open("reddit_data.csv", "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Text", "Date", "Section"])  # Column headers
    writer.writerows(data)  # Write the data rows

print("Data saved to reddit_data.csv!")

Data saved to reddit_data.csv!


In [27]:
data = pd.read_csv('reddit_data.csv')

In [34]:
data.iloc[2]

Text    [The animation looks cool](https://www.apple.c...
Date                                      24 October 2023
Name: 2, dtype: object

In [38]:
data.iloc[9]['Text']

'Apple hyping speed so there’s nothing else likely to be changing I imagine.'

In [26]:
df=pd.DataFrame(data)

NameError: name 'data' is not defined

In [66]:
df

,0,1
0,Apple mouth headass\n,30 October 2023
1,This is a reminder for people not to post poli...,30 October 2023
2,"Capital gums, lower case teeth",30 October 2023
3,I know a girl who had gums like this and she g...,30 October 2023
4,"Ok, mean story time. I know this girl who work...",30 October 2023
...,...,...
3364,"So then you think\n\n""Phone calls are TOP TIER...",30 October 2023
3365,"Sorry, I didn't realise that you don't underst...",30 October 2023
3366,Yeah. Because we all know tiers are only perso...,30 October 2023
3367,>Yeah. Because we all know tiers are only pers...,30 October 2023


In [12]:
# Separate the last 20 columns and the rest
last_20_cols = df.columns[-1:].tolist()
other_cols = df.columns[:-1].tolist()

# Concatenate the lists of column names
new_order = last_20_cols + other_cols

# Re-index the dataframe
df[new_order]


NameError: name 'df' is not defined

In [69]:
df

,0,1
0,Apple mouth headass\n,30 October 2023
1,This is a reminder for people not to post poli...,30 October 2023
2,"Capital gums, lower case teeth",30 October 2023
3,I know a girl who had gums like this and she g...,30 October 2023
4,"Ok, mean story time. I know this girl who work...",30 October 2023
...,...,...
3364,"So then you think\n\n""Phone calls are TOP TIER...",30 October 2023
3365,"Sorry, I didn't realise that you don't underst...",30 October 2023
3366,Yeah. Because we all know tiers are only perso...,30 October 2023
3367,>Yeah. Because we all know tiers are only pers...,30 October 2023
